## Number 1 

In [1]:
import pandas as pd

from collections import Counter
import math

data1 = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
df1 = pd.read_csv(data1,header=None, encoding='utf-8')

df1.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [2]:

make = {'alfa-romero': 0,'audi': 1,'bmw': 2,'chevrolet': 3,'dodge': 4,'honda': 5,'isuzu': 6,
        'jaguar': 7,'mazda': 8,'mercedes-benz':9, 'mercury':10, 'mitsubishi': 11,'nissan':12,
        'peugot':13,'plymouth': 14,'porsche': 15, 'renault': 16, "saab": 17,'subaru': 18,
        'toyota': 19,'volkswagen': 20,'volvo': 21
}

fuel_type = {'diesel': 0, 'gas':1}
aspiration = {'std':0,'turbo':1}
num_doors = {'four':0, 'two':1, '?':2}
body_style = {'hardtop':0, 'wagon':1, 'sedan':2, 'hatchback':3, 'convertible':4}
drive_wheels = {'4wd':0, 'fwd':1, 'rwd':2}
engine_location = {'front':0, 'rear':1}
engine_type = {'dohc':0, 'dohcv':1, 'l':2, 'ohc':3, 'ohcf':4, 'ohcv':5,'rotor':6}
num_cylinders = {'eight':0, 'five':1, 'four':2, 'six':3, 'three':4, 'twelve':5, 'two':6}
fuel_system = {'1bbl':0, '2bbl':1, '4bbl':2, 'idi':3, 'mfi':4, 'mpfi':5, 'spdi':6, 'spfi':7}

def transform_data(values):
    for (index, value) in enumerate(values):
        if values[index][1] == '?' : values[index][1] = 0
        else: values[index][1] = int(values[index][1])
        values[index][2] = make[value[2]]
        values[index][3] = fuel_type[value[3]]
        values[index][4] = aspiration[value[4]]
        values[index][5] = num_doors[value[5]]
        values[index][6] = body_style[value[6]]
        values[index][7] = drive_wheels[value[7]]
        values[index][8] = engine_location[value[8]]
        values[index][14] = engine_type[value[14]]
        values[index][15] = num_cylinders[value[15]]
        values[index][17] = fuel_system[value[17]]
        
        if values[index][18] == '?': values[index][18] = 0
        else: values[index][18] = float(values[index][18])
            
        if values[index][19] == '?': values[index][19] = 0
        else: values[index][19] = float(values[index][19])
            
        if values[index][21] == '?': values[index][21] = 0
        else: values[index][21] = float(values[index][21])
            
        if values[index][22] == '?': values[index][22] = 0
        else: values[index][22] = float(values[index][22])
            
        if values[index][25] == '?': values[index][25] = 0
        else: values[index][25] = float(values[index][25])

In [3]:
def knn(data, query, k, distance_fn, choice_fn):
    neighbor_distances_and_indices = []
    
    # For each example in the data
    for index, example in enumerate(data):
        # Calculate the distance between the query example 
        # and the current example from the data.
        distance = distance_fn(example[:-1], query)
        
        # Add the distance and the index of the example to an 
        # ordered collection
        neighbor_distances_and_indices.append((distance, index))
    
    # Sort the ordered collection of distances and indices from
    # smallest to largest (in ascending order) by the distances
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    
    # Pick the first K entries from the sorted collection
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    
    # Get the labels of the selected K entries
    k_nearest_labels = [data[i][-1] for distance, i in k_nearest_distances_and_indices]
    
    print("k_nearest_labels=", k_nearest_labels) 
    # 7. If regression (choice_fn = mean), return the average 
    # of the K labels
    # 8. If classification (choice_fn = mode), return the mode of 
    # the K labels
    return k_nearest_distances_and_indices , choice_fn(k_nearest_labels)

In [4]:
def mean(labels):
    return sum(labels) / len(labels)

In [5]:
def mode(labels):
    return Counter(labels).most_common(1)[0][0]

In [6]:
def euclidean_distance(point1, point2): 
    sum_squared_distance = 0
    for i in range(len(point1)):
        sum_squared_distance += math.pow(point1[i] - point2[i], 2) 
    return math.sqrt(sum_squared_distance)

In [7]:
def recommender(k, query):
    values = df1.to_numpy()
    transform_data(values)
    k_nearest_neighbors, prediction = knn(values, query, k, euclidean_distance, mode)
    nearest_index = k_nearest_neighbors[0][1]
    return getOriginalData(values[nearest_index])

def getOriginalData(value):
    if value[1] == 0: value[1] = '?'
    value[2] = list(make.keys())[list(make.values()).index(value[2])]
    value[3] = list(fuel_type.keys())[list(fuel_type.values()).index(value[3])]
    value[4] = list(aspiration.keys())[list(aspiration.values()).index(value[4])]
    value[5] = list(num_doors.keys())[list(num_doors.values()).index(value[5])]
    value[6] = list(body_style.keys())[list(body_style.values()).index(value[6])]
    value[7] = list(drive_wheels.keys())[list(drive_wheels.values()).index(value[7])]
    value[8] = list(engine_location.keys())[list(engine_location.values()).index(value[8])]
    value[14] = list(engine_type.keys())[list(engine_type.values()).index(value[14])]
    value[15] = list(num_cylinders.keys())[list(num_cylinders.values()).index(value[15])]
    value[17] = list(fuel_system.keys())[list(fuel_system.values()).index(value[17])]
    if value[18] == 0: value[18] = '?'
    if value[19] == 0: value[19] = '?'
    if value[21] == 0: value[21] = '?'
    if value[22] == 0: value[22] = '?'
    if value[25] == 0: value[25] = '?'
    return value

In [9]:
query = [1, 100, make['bmw'], fuel_type['gas'], aspiration['std'], num_doors['four'],
         body_style['sedan'], drive_wheels['fwd'], engine_location['front'], 100.0,
         170.0, 65.0, 50.0, 2000, engine_type['rotor'], num_cylinders ['four'], 200,
         fuel_system['mfi'], 3.0, 3.0, 15, 200, 5000, 23, 27, 30000]

five_rec = recommender(5, query)

ten_rec = recommender(10,query)
         
print(five_rec)
print(ten_rec)

k_nearest_labels= [7395.0, 6695.0, 6795.0, 6095.0, 5118.0]
k_nearest_labels= [7395.0, 6695.0, 6795.0, 6095.0, 5118.0, 5195.0, 7349.0, 7499.0, 7999.0, 7799.0]
[1 113 'mazda' 'gas' 'std' 'four' 'sedan' 'fwd' 'front' 93.1 166.8 64.2
 54.1 1950 'ohc' 'four' 91 '2bbl' 3.08 3.15 9.0 68.0 5000.0 31 38 7395.0]
[1 113 'mazda' 'gas' 'std' 'four' 'sedan' 'fwd' 'front' 93.1 166.8 64.2
 54.1 1950 'ohc' 'four' 91 '2bbl' 3.08 3.15 9.0 68.0 5000.0 31 38 7395.0]
